<a href="https://colab.research.google.com/github/akamojo/PAD-project/blob/master/part2v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 604kB 3.4MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-fn9uchxd/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import re
from tqdm import tqdm_notebook as tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/en1.8m.txt", "r", encoding="utf8")
#file = open("en1.8m.txt", "r", encoding="utf8")

whole_text = file.read()
text = whole_text.split('\n')

In [0]:
def append_space(text, pattern, left=True):
    p = re.compile("[" + pattern + "]")
    
    for i, m in enumerate(p.finditer(text)):
        if left:
            text = text[: m.start() + i] + " " + text[m.start() + i :]
        else:
            text = text[: m.start() + 1 + i] + " " + text[m.start() + 1 + i :]
            
    return text

In [6]:
for i in range(len(text)):
    text[i] = append_space(text[i], ",\])>")
    text[i] = append_space(text[i], ".")
    text[i] = append_space(text[i], "\[(<", False)
    
test = text[0]
test

"Greek Christian scribes played a crucial role in the preservation of Aristotle by copying all the extant Greek language manuscripts of the corpus . The first Greek Christians to comment extensively on Aristotle were John Philoponus , Elias , and David in the sixth century , and Stephen of Alexandria in the early seventh century . John Philoponus stands out for having attempted a fundamental critique of Aristotle's views on the eternity of the world , movement , and other elements of Aristotelian thought . After a hiatus of several centuries , formal commentary by Eustratius and Michael of Ephesus reappears in the late eleventh and early twelfth centuries , apparently sponsored by Anna Comnena ."

In [0]:
def build_n_gram(text_arr, n_gram, grams):
    cur_gram = []
    count_gram = 0

    for i in range(len(text_arr)):

        cur_gram = [text_arr[i]]

        for j in range(i + 1, i + n_gram):
            if j < len(text_arr):
                cur_gram.append(text_arr[j])

        if len(cur_gram) == n_gram:
            try:
                grams[" ".join(cur_gram)] += 1
            except KeyError:
                grams[" ".join(cur_gram)] = 1

    return grams

In [0]:
def build_dict(text, n_s):
    dictionary = []
    for n in range(n_s):
        dictionary.append(dict())
    
    for n in range(n_s):
        for line in text:
            dictionary[n] = build_n_gram(line.split(), n + 1, dictionary[n])
        
    return dictionary

In [0]:
def SCP_f(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] ** 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] * dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [0]:
def dice(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] * 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] + dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [0]:
def LocalMax(text, table):
    xys = dict()
    
    for nr in tqdm(range(len(text))):
        line = text[nr]

        text_arr = line.split()

        if len(text_arr) < 2:
            continue

        for i in range(len(text_arr) - 2):
            cur_gram = " ".join(text_arr[i:i+2])

            j = i + 3
            n_gram = 1

            while j < len(text_arr) and n_gram < 6:
                next_gram = " ".join(text_arr[i:j])

                try:
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]

                cur_gram = next_gram

                j += 1
                n_gram += 1

            cur_gram = " ".join(text_arr[i:i+2])
            j = i - 1
            n_gram = 1

            while j >= 0 and n_gram < 6:
                next_gram = " ".join(text_arr[j:i+2])


                try:                        
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]


                cur_gram = next_gram

                j -= 1
                n_gram += 1
              
    return xys

In [0]:
def main(text, max_gram, threshold, glue_fun):
    print("Building dictionary with frequencies")

    dictionary = build_dict(text, max_gram)
    
    table = dict()
        
    print("Proceeding to calculating glue value")
    print("Glue value calculated for:")

    for i in range(2, max_gram + 1):
        print(i, "grams")
        for s in dictionary[i - 1]:
          table[s] = glue_fun(s, dictionary) #dice(s, dictionary)
    
    print("Proceeding to Localmax")
    
    RE = []
    xys = LocalMax(text, table)
            
    for g in xys:
        if xys[g][0] == -1:
            xys[g][0] = xys[g][1]
        
        val = (xys[g][0] + xys[g][1]) / 2
        
        if table[g] > val and re.match(r"^[a-z\'A-Z\s]*$", g) and dictionary[len(g.split()) - 1][g] > threshold:
            RE.append(g)
    
    return RE

In [34]:
RE = main([test], 7, 1, SCP_f)
print("Obtained", len(RE), "Relevant Expressions")
RE

Building dictionary with frequencies
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams
Proceeding to Localmax



Obtained 2 Relevant Expressions


['in the', 'John Philoponus']

In [0]:
RE = main(text, 7)
print("Obtained", len(RE), "Relevant Expressions")

Building dictionary with frequencies
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams


  0%|          | 67/69468 [00:00<01:44, 664.87it/s]

Proceeding to Localmax


100%|██████████| 69468/69468 [00:51<00:00, 1352.82it/s]


Obtained 38124 Relevant Expressions


In [0]:
RE = main(text, 7)
print("Obtained", len(RE), "Relevant Expressions")

Building dictionary with frequencies
Proceeding to calculating glue value
Glue value calculated for:
2 grams
3 grams
4 grams
5 grams
6 grams
7 grams


  0%|          | 83/69468 [00:00<01:23, 827.34it/s]

Proceeding to Localmax


100%|██████████| 69468/69468 [00:53<00:00, 1309.12it/s]


Obtained 19059 Relevant Expressions


In [0]:
RE[:100]

['crucial role',
 'in the',
 'of the',
 'The first',
 'John Philoponus',
 'on the',
 'and other',
 'communications equipment',
 'based on',
 'with the',
 'divided into',
 'the use of',
 'the study of',
 'known as',
 'what is now',
 'from the',
 'to the',
 'as a',
 'was not',
 'end of',
 'by the',
 'of their',
 'After independence',
 'civil war',
 'such as',
 'the dawn of',
 'the dawn of the',
 'Ottoman Empire',
 'the geopolitical',
 'He also',
 'for the',
 'western Europe',
 'as well as',
 'in the form of',
 'is a',
 'Indic scripts',
 'Southeast Asia',
 'they are',
 'Northern India',
 'Sri Lanka',
 'the site of',
 'Charles VI of France',
 'VI of France',
 'as the',
 'According to',
 'that they',
 'they were',
 'the start of',
 'start of the battle',
 'to be',
 'formula for calculating',
 'is not',
 'Western Church',
 'abbots of monasteries',
 'to a',
 'as an',
 'Roman Catholic',
 'Catholic Church',
 'Orthodox Church',
 'are permitted to be',
 'the rank of',
 'there are',
 'are not',
 '